In [ ]:
using Zygote, StaticArrays, LinearAlgebra, Kronecker, LsqFit
import Zygote.hessian

Versions [Pluto](https://github.com/vlc1/ene-4102c-td/blob/master/td5.jl) et [Jupyter](https://vlc1.github.io/ene-4102c/td5/sujet.ipynb) de ce notebook.


# Method of Manufactured Solutions

On s'intéresse ici à la résolution du problème de conduction dans une plaque $\left ( x, y \right ) \in \left [ 0, 1 \right ] ^ 2$ en régime stationnaire. On se propose donc de résoudre le problème suivant par la méthode des différences finies :
$$
0 = \Delta \theta \left ( x, y \right ) + \omega \left ( x, y \right )
$$
où le Laplacien en coordonnées Cartésiennes, c'est à dire
$$
\Delta \theta = \left . \frac{\partial ^ 2 \theta}{\partial x ^ 2} \right \vert _ y +  \left . \frac{\partial ^ 2 \theta}{\partial y ^ 2} \right \vert _ x.
$$

Le champs de température $\theta$ sera soumis aux conditions aux limites suivantes :
$$
\left \{ \begin{aligned}
\partial_x \theta \left ( 0, y \right ) & = g_1 \left ( y \right ), \\
\theta \left ( 1, y \right ) & = \theta_1 \left ( y \right ),
\end{aligned} \right . \quad \mathrm{and} \quad \left \{ \begin{aligned}
\partial_y \theta \left ( x, 0 \right ) & = g_2 \left ( x \right ), \\
\theta \left ( x, 1 \right ) & = \theta_2 \left ( x \right ).
\end{aligned} \right .
$$

On se propose d'employer la *Method of Manufactured Solutions* afin de vérifier l'implémentation. Étant donnée une solution $\theta$ choisie à l'avance, l'idée est de

* Déterminer $\omega$, $g_1$, $g_2$, $\theta_1$ et $\theta_2$ à partir de cette solution,
* Résoudre numériquement le problème et comparer solution numérique à la solution choisie afin d'obtenir l'ordre de la méthode.

1. Calculer $\omega$, $g_1$, $g_2$, $\theta_1$ et $\theta_2$ correspondant à la solution
$$
\theta \colon \left ( x, y \right ) \mapsto \sin \left ( \pi x \right ) \cos \left (\pi y \right ).
$$
2. Les fonctions `Δ`, `left`, `bottom`, `right` et `top` prennent $\theta$ comme argument et retournent respectivement $\omega$, $g_1$, $g_2$, $\theta_1$ et $\theta_2$ automatiquement grâce à la bibliothèque d'*Algorithmic Differentiation* `Zygote.jl`. Vérifier graphiquement que ces fonctions retournent bien le résultat attendu.


In [ ]:
# Question 2
θ(x, y) = x + y
ω(x, y) = zero(x) + zero(y)
g₁(y) = one(y)
g₂(x) = one(x)
θ₁(y) = y
θ₂(x) = x

In [ ]:
# Question 2 - NE PAS MODIFIER
θ(x::AbstractVector) = θ(x[1], x[2])
Δ(f) = (x, y) -> first(hessian(f, SVector(x, y))) + last(hessian(f, SVector(x, y)))
left(f) = y -> first(first(gradient(f, SVector(zero(y), y))))
bottom(f) = x -> last(first(gradient(f, SVector(x, zero(x)))))
right(f) = y -> f(one(y), y)
top(f) = x -> f(x, one(x))

3. Implémenter la fonction `exact` qui évalue $θ$ à chaque point du maillage.


In [ ]:
# Question 3
function exact(f, n)
    zeros(n...)
end

4. Les fonctions `phi`, `spacing` et `laplacian` du TD4 sont rappelées plus bas. S'en servir pour implémenter la discrétisation du laplacien en deux dimensions (voir le [produit de Kronecker](https://fr.wikipedia.org/wiki/Produit_de_Kronecker) implémenté par la fonction `kron` de `Kronecker.jl`).


In [ ]:
# Question 4
function laplacian(n::NTuple{2, Int})
    id = Diagonal.(fill.(-1.0, n))
    kron(id[2], id[1]) + kron(id[2], id[1])
end

In [ ]:
# Question 4 - NE PAS MODIFIER
phi() = 1 / √3

In [ ]:
# Question 4 - NE PAS MODIFIER
spacing(n) = 1 / (n + phi())

In [ ]:
# Question 4 - NE PAS MODIFIER
function laplacian(n)
    h = spacing(n)

    A = Tridiagonal(zeros.((n - 1, n, n - 1))...)

    # gauche
    A[1, 1] = 1 / (phi() + 1 / 2) / h ^ 2
    A[1, 2] = -1 / (phi() + 1 / 2) / h ^ 2

    # intérieur
    for j in 2:n - 1
        A[j, j - 1] = -1 / h ^ 2
        A[j, j] = 2 / h ^ 2
        A[j, j + 1] = -1 / h ^ 2
    end

    # droite
    A[n, n - 1] = -1 / h ^ 2
    A[n, n] = 2 / h ^ 2

    A
end

5. La fonction `mesh` du TD4 est rappelée plus bas. S'en servir ainsi que `Δ`, `left`, `bottom`, `right` et `top` pour implémenter la fonction `rhs` qui assemble le second membre.


In [ ]:
# Question 5
function rhs(f, n)
    zeros(n...)
end

In [ ]:
# Question 5 - NE PAS MODIFIER
mesh(n) = [spacing(n) * (phi() + (j - 1)) for j in 1:n]

6. La fonction `numerical` définie plus bas résoudre numériquement le problème. Implémenter la fonction `error` ci-dessous qui calcule la norme de la différence entre la solution numérique et la solution analytique. Vérifier que celle-ci est nulle lorsque $\theta$ est une fonction quadratique.


In [ ]:
# Question 6
error(f, n) = zero(Float64)

In [ ]:
# Question 6 - NE PAS MODIFIER
function numerical(f, n)
    A = laplacian(n)
    b = reshape(rhs(f, n), prod(n))

    x = A \ b

    reshape(x, n...)
end

7. Pour la solution de la question 1, vérifier le schéma est bien d'ordre deux.
